# Imports

In [10]:
import os
import math
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter

import numpy as np

# Constants

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
BATCH_SIZE = 64
NUM_WORKERS = 0

In [14]:
ROOT_DIR = Path('.')
DATA_DIR = ROOT_DIR / 'data'
REPORTS_DIR = ROOT_DIR / 'reports'
MODELS_DIR = REPORTS_DIR / 'models'
RESULTS_DIR = REPORTS_DIR / 'results'
RUNS_DIR = REPORTS_DIR / 'runs'

# Data loading

In [4]:
train_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5)]
)
val_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5)]
)
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]
)

In [ ]:
train_proportion = 0.9
num_train = 50000

indices = list(range(num_train))
split = int(np.floor(train_proportion * num_train))
np.random.shuffle(indices)

train_idx, val_idx = indices[:split], indices[split:]
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [ ]:

train_dataset = datasets.CIFAR10(root=DATA_DIR, train=True, 
                                 download=True, transform=train_transform)

val_dataset = datasets.CIFAR10(root=DATA_DIR, train=True, 
                               download=True, transform=val_transform)

test_dataset = datasets.CIFAR10(root=DATA_DIR, train=False,
                                download=True, transform=test_transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, 
                          sampler=train_sampler, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, 
                        sampler=val_sampler, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                         num_workers=NUM_WORKERS)

# Layers

In [ ]:
class OWALayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.use_bias = bias  # Separate flag
        self.weights = nn.Parameter(torch.empty((out_features, in_features)))
        if self.use_bias:
            self.bias = nn.Parameter(torch.empty(out_features))
        else:
            self.register_parameter('bias', None)
        self.init_parameters()
        
    def init_parameters(self):
        stdv = 1. / math.sqrt(self.weights.size(1))
        nn.init.uniform_(self.weights, -stdv, stdv)
        if self.bias is not None:
            nn.init.uniform_(self.bias, -stdv, stdv)
            
    def forward(self, x):
        # Specify the dimension to sort along, typically the feature dimension
        x_sorted, _ = torch.sort(x, dim=1, descending=True)
        normalized_weights = F.softmax(self.weights, dim=1)
        output = torch.matmul(x_sorted, normalized_weights.t())
        if self.bias is not None:
            output += self.bias
        return output

In [ ]:
class ChoquetLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(ChoquetLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.use_bias = bias

        self.v = nn.Parameter(torch.empty(out_features, in_features))

        if self.use_bias:
            self.bias = nn.Parameter(torch.empty(out_features))
        else:
            self.register_parameter('bias', None)

        self.init_parameters()

    def init_parameters(self):
        stdv = 1. / math.sqrt(self.in_features)
        nn.init.uniform_(self.v, -stdv, stdv)
        if self.use_bias:
            nn.init.uniform_(self.bias, -stdv, stdv)

    def forward(self, x):
        if x.dim() != 2:
            raise ValueError("Input tensor must be 2D (batch_size, in_features)")

        x_sorted, _ = torch.sort(x, dim=1, descending=True)
        v_padded = F.pad(self.v, (0, 1), "constant", 0)
        delta_v = v_padded[:, :-1] - v_padded[:, 1:]
        x_sorted_expanded = x_sorted.unsqueeze(1)  # (batch_size, 1, in_features)
        delta_v_expanded = delta_v.unsqueeze(0)  # (1, out_features, in_features)
        output = torch.sum(x_sorted_expanded * delta_v_expanded, dim=2)

        if self.use_bias:
            output += self.bias

        return output


In [ ]:
# TODO: finish implementing this
class AggPoolingLayer(nn.Module):
    def __init__(self, kernel_size, stride, padding):
        super(AggPoolingLayer, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        raise NotImplementedError()
        
    def forward(self, x):
        x = F.pad(
            x,
            (self.padding, self.padding, self.padding, self.padding),
            mode="constant",
            value=0,
        )
        x_shape = x.shape
        x = x.unfold(2, self.kernel_size, self.stride).unfold(
            3, self.kernel_size, self.stride
        )  # (batch_size, in_channels, num_patches_height, num_patches_width, kernel_size, kernel_size)
        x = x.reshape(
            x_shape[0],
            x_shape[1],
            x_shape[2]//self.stride,
            x_shape[3]//self.stride,
            self.kernel_size**2,
        )  # (batch_size, in_channels, img_height, img_width, kernel_size*kernel_size) for each pixel there are kernel_size**2 values
        x, _ = x.max(dim=4)
        return x

# Model

In [ ]:
class LeNetModel(nn.Module):

    def __init__(self, conv_filters=[64, 64], linear_sizes=[384, 192], num_classes=10):
        super(LeNetModel, self).__init__()
        self.conv1 = nn.Conv2d(3, conv_filters[0], [2,2], [1,1])
        self.pool1 = nn.MaxPool2d([2,2], [2,2])
        self.conv2 = nn.Conv2d(conv_filters[0], conv_filters[1], [2,2], [1,1])
        self.pool2 = nn.MaxPool2d([2,2], [2,2])
        self.fc1 = nn.Linear(conv_filters[1]*7*7, linear_sizes[0])       
        self.fc2 = nn.Linear(linear_sizes[0], linear_sizes[1])
        self.fc3 = nn.Linear(linear_sizes[1], num_classes)
        
    def forward(self, x: torch.Tensor):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Training

In [ ]:
writer = SummaryWriter(log_dir=RUNS_DIR)

In [ ]:
def train(model, train_loader, criterion, optimizer, val_loader=None, num_epochs=20, device='cuda'):
    train_acc = []  
    train_loss = []
    if val_loader is not None:
        val_acc = []
        val_loss = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        count_evaluated = 0
        count_correct = 0
        
        for batch_idx, data in enumerate(train_loader, 0):    
            model.train()  
            inputs, labels = data[0].to(device), data[1].to(device)  
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            count_evaluated += inputs.shape[0]
            count_correct += torch.sum(labels == torch.max(outputs, dim=1)[1])
            
        print('Training: [%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / (batch_idx+1)))
        
        train_loss.append(running_loss / (batch_idx+1))
        train_acc.append(float(count_correct) / count_evaluated)
        
        if val_loader is not None:
            running_loss_val = 0.0
            count_evaluated = 0
            count_correct = 0
            model.eval()
            
            with torch.no_grad():
                for val_batch_idx, data_val in enumerate(val_loader, 0):
                    inputs_val, labels_val = data_val[0].to(device), data_val[1].to(device)
                    outputs_val = model(inputs_val)
                    loss = criterion(outputs_val, labels_val)
                    running_loss_val += loss.item()
                    count_evaluated += inputs_val.shape[0]
                    count_correct += torch.sum(labels_val == torch.max(outputs_val, dim=1)[1])
                    
                val_loss.append(running_loss_val / (val_batch_idx + 1))
                acc_val = float(count_correct) / count_evaluated
                
                print('Validation: epoch %d - acc: %.3f' %
                            (epoch + 1, acc_val))
                val_acc.append(acc_val)
                
        # Tensorboard
        writer.add_scalar('Loss/Validation', val_loss[-1], global_step=epoch)
        writer.add_scalar('Accuracy/Validation', val_acc[-1], global_step=epoch)
        writer.add_scalar('Loss/Train', train_loss[-1], global_step=epoch)
        writer.add_scalar('Accuracy/Train', train_acc[-1], global_step=epoch)
        for name, param in model.named_parameters():
            writer.add_histogram(f"Parameters/{name}", param, epoch)
            if param.grad is not None:
                writer.add_histogram(f"Gradients/{name}", param.grad, epoch)
                
    return model

# Testing

In [ ]:
def test(model, test_loader, criterion, device='cuda'):
    with torch.no_grad():
        number_samples = 0
        number_correct = 0
        running_loss_test = 0.0
        for test_batch_idx, data_test in enumerate(test_loader, 0):
            inputs_test, labels_test = data_test[0].to(device), data_test[1].long().to(device)
            outputs_test = model(inputs_test)
            loss = criterion(outputs_test, labels_test)
            running_loss_test += loss.cpu().numpy()
            
            _, outputs_class = torch.max(outputs_test, dim=1)
            number_correct += torch.sum(outputs_class == labels_test).cpu().numpy()
            number_samples += len(labels_test)
            
        acc_test = number_correct / number_samples
        
        print('Test - Accuracy: %.3f' % acc_test)
        print('Test - CrossEntropy: %.3f' % (running_loss_test / (test_batch_idx+1)))